In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('predykcja').getOrCreate()

In [3]:
data = spark.read.csv(r"C:\Users\Asus\Downloads\train_student.csv", inferSchema = True, header=True)

In [25]:
import statsmodels.api as sm

X = data['Sex', 'LicenseYear', 'CarBrand', 'CarType', 'CarEngine', 'EngineCap', 'CarValue', 'AssistanceYears', 'Accidents', 'Age', 'CarAge']
y = data['NextAccident']


X = sm.add_constant(X)


model = sm.Logit(y, X)
result = model.fit()
result.summary()

TypeError: unsupported operand type(s) for -: 'DataFrame' and 'DataFrame'

In [5]:
from pyspark.sql.functions import col, cast

In [9]:
data = data.withColumn('Accidents', data['Accidents'].cast('int'))

In [7]:
data.printSchema

<bound method DataFrame.printSchema of DataFrame[BirthYear: string, Sex: int, LicenseYear: double, CarBrand: int, CarType: int, CarYear: int, CarEngine: int, EngineCap: int, CarValue: int, AssistanceYears: double, Accidents: string, NextAccident: int]>

In [7]:
data.head(5)

[Row(BirthYear='1987', Sex=0, LicenseYear=6.1642, CarBrand=9, CarType=4, CarYear=2012, CarEngine=1782, EngineCap='124', CarValue=61400, AssistanceYears=1.3328, Accidents=0, NextAccident=1),
 Row(BirthYear='1976', Sex=1, LicenseYear=11.4121, CarBrand=3, CarType=2, CarYear=2012, CarEngine=1246, EngineCap='147', CarValue=60250, AssistanceYears=2.6656, Accidents=0, NextAccident=0),
 Row(BirthYear='1970', Sex=1, LicenseYear=15.7437, CarBrand=3, CarType=2, CarYear=2013, CarEngine=1318, EngineCap='120', CarValue=61200, AssistanceYears=12.5783, Accidents=0, NextAccident=0),
 Row(BirthYear='1976', Sex=0, LicenseYear=23.0741, CarBrand=8, CarType=1, CarYear=1998, CarEngine=1766, EngineCap='NA', CarValue=33600, AssistanceYears=14.161, Accidents=1, NextAccident=0),
 Row(BirthYear='1954', Sex=0, LicenseYear=12.3284, CarBrand=3, CarType=1, CarYear=2013, CarEngine=1908, EngineCap='184', CarValue=44450, AssistanceYears=11.662, Accidents=None, NextAccident=0)]

In [11]:
columns = ['BirthYear',
 'Sex',
 'LicenseYear',
 'CarBrand',
 'CarType',
 'CarYear',
 'CarEngine',
 'EngineCap',
 'CarValue',
 'AssistanceYears',
 'Accidents',
 'NextAccident']

In [15]:
import statsmodels.api as sm

In [12]:
from pyspark.ml.feature import Imputer

In [13]:
for column in columns:
    imputer = Imputer(inputCols=[column], outputCols=[column])
    model = imputer.fit(data)
    data = model.transform(data)

In [14]:
current = 2024
data = data.withColumn('Age', current - col('BirthYear'))

In [15]:
data = data.withColumn('CarAge', current - col('CarYear'))

In [23]:
data = data.drop('CarYear')

In [27]:
train_set, test_set = data.randomSplit([0.7, 0.3])

In [28]:
from pyspark.ml.classification import LogisticRegression

In [29]:
from pyspark.ml.feature import VectorAssembler

In [24]:
data

DataFrame[Sex: int, LicenseYear: double, CarBrand: int, CarType: int, CarEngine: int, EngineCap: int, CarValue: int, AssistanceYears: double, Accidents: int, NextAccident: int, Age: int, CarAge: int]

In [31]:
from pyspark.ml.stat import Correlation

In [36]:
assembler = VectorAssembler(inputCols=[
 'LicenseYear',
'CarEngine',
'EngineCap',
'CarValue'], outputCol='features')

VectorAssembler_bb9ed408a258

array([[ 1.00000000e+00, -1.07113334e-03,  1.10977450e-02,
        -2.20497839e-02, -1.46693354e-01, -9.93984707e-02,
        -2.27020267e-02, -1.53858756e-03, -2.06875994e-03,
        -5.08790038e-03,  3.06542624e-02],
       [-1.07113334e-03,  1.00000000e+00, -5.70375706e-02,
        -9.93217877e-03, -3.83479751e-02, -2.93160406e-02,
        -1.48184237e-02,  7.49793846e-01,  3.24680809e-03,
         7.35521814e-01,  2.34859646e-02],
       [ 1.10977450e-02, -5.70375706e-02,  1.00000000e+00,
         9.26967091e-03,  6.05231782e-03,  5.63920219e-03,
         4.59341421e-04, -4.38516383e-02, -4.32172436e-03,
        -4.43491472e-02, -6.76680261e-03],
       [-2.20497839e-02, -9.93217877e-03,  9.26967091e-03,
         1.00000000e+00, -5.75181983e-02, -3.42061878e-02,
        -1.76041905e-02, -7.17657690e-03, -7.23376847e-04,
        -7.75246954e-03,  2.22968563e-02],
       [-1.46693354e-01, -3.83479751e-02,  6.05231782e-03,
        -5.75181983e-02,  1.00000000e+00,  5.88013667e-01,
  

In [62]:
import statsmodels as sm

In [64]:
from scipy.stats import ttest_ind

TypeError: 'DataFrame' object does not support item assignment

In [37]:
from pyspark.ml.feature import StandardScaler

In [38]:
scaler = StandardScaler(inputCol='features', outputCol='features_scaled')

In [39]:
lr =LogisticRegression(labelCol='NextAccident', featuresCol='features_scaled')

In [40]:
from pyspark.ml import Pipeline

In [41]:
pip = Pipeline(stages=[assembler, scaler, lr])

In [42]:
model = pip.fit(train_set)

In [43]:
predictions = model.transform(test_set)

In [50]:
predictions.select("NextAccident", "prediction", "probability").show(50)

+------------+----------+--------------------+
|NextAccident|prediction|         probability|
+------------+----------+--------------------+
|           0|       0.0|[0.76558613193194...|
|           0|       0.0|[0.90215025965139...|
|           0|       0.0|[0.85737945803063...|
|           0|       0.0|[0.88839065706620...|
|           0|       0.0|[0.83224303816519...|
|           0|       0.0|[0.84463997209849...|
|           0|       0.0|[0.77893614056514...|
|           0|       0.0|[0.84764979955390...|
|           0|       0.0|[0.81579059056015...|
|           1|       0.0|[0.83630461602620...|
|           0|       0.0|[0.84816906287562...|
|           1|       0.0|[0.85799314069787...|
|           0|       0.0|[0.88522858210598...|
|           0|       0.0|[0.85605000321307...|
|           0|       0.0|[0.90475209751041...|
|           0|       0.0|[0.85124720374369...|
|           0|       0.0|[0.84366835312275...|
|           0|       0.0|[0.87302522812366...|
|           1

In [46]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [47]:
evaluator = BinaryClassificationEvaluator(labelCol='NextAccident')

In [48]:
accuracy = evaluator.evaluate(predictions)

In [49]:
accuracy

0.5985279379475762

In [51]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

crossval = CrossValidator(estimator=pip,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)  


cvModel = crossval.fit(train_set)

predictions = cvModel.transform(test_set)

predictions.select("NextAccident", "prediction", "probability").show()


accuracy = evaluator.evaluate(predictions)



+------------+----------+--------------------+
|NextAccident|prediction|         probability|
+------------+----------+--------------------+
|           0|       0.0|[0.79334105571168...|
|           0|       0.0|[0.87460279207649...|
|           0|       0.0|[0.84534691841221...|
|           0|       0.0|[0.86654737734876...|
|           0|       0.0|[0.83255815522284...|
|           0|       0.0|[0.83534175420084...|
|           0|       0.0|[0.80104284563944...|
|           0|       0.0|[0.84116065509066...|
|           0|       0.0|[0.82391015388723...|
|           1|       0.0|[0.83380055421351...|
|           0|       0.0|[0.84039265142157...|
|           1|       0.0|[0.84710740757723...|
|           0|       0.0|[0.86447056035618...|
|           0|       0.0|[0.84400165558775...|
|           0|       0.0|[0.87696146329227...|
|           0|       0.0|[0.84261123491783...|
|           0|       0.0|[0.83935460574488...|
|           0|       0.0|[0.85585419959019...|
|           1

In [59]:
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
import numpy as np

def plot_learning_curve(estimator, title, X, y, cv=None, n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")

    plt.legend(loc="best")
    return plt


In [68]:
data.corr('CarType', 'NextAccident')

-0.002003246380069838

In [63]:
data.columns

['BirthYear',
 'Sex',
 'LicenseYear',
 'CarBrand',
 'CarType',
 'CarYear',
 'CarEngine',
 'EngineCap',
 'CarValue',
 'AssistanceYears',
 'Accidents',
 'NextAccident',
 'Age',
 'CarAge']

In [71]:
for column in data.columns:
    cor = data.corr(column, 'NextAccident')
    print(cor)

0.0167904799370922
-0.0027457531231849393
-0.017710672366369175
0.0010865169992085623
-0.002003246380069838
-0.08079373302229763
0.03599980627411257
0.029860973045400436
-0.11722247594154574
-0.01345153269974616
0.0027975992951380447
1.0
-0.016790479937093393
0.08079373302229692


In [74]:
import statsmodels.api as sm

# X to macierz cech (bez stałej), y to wektor zmiennych zależnych
X = data['Sex', 'LicenseYear', 'CarBrand', 'CarType', 'CarEngine', 'EngineCap', 'CarValue', 'AssistanceYears', 'Accidents', 'Age', 'CarAge']
y = data['NextAccident']

# Dodanie stałej do macierzy cech (jeśli chcesz ocenić istotność stałej)
X = sm.add_constant(X)

# Utworzenie modelu regresji logistycznej
model = sm.Logit(y, X)

# Dopasowanie modelu
result = model.fit()

# Wypisanie podsumowania modelu
print(result.summary())


TypeError: unsupported operand type(s) for -: 'DataFrame' and 'DataFrame'